In [1]:
import pymongo
import pandas as pd


In [2]:
db = pymongo.MongoClient().scraper_test_dev

In [12]:
platcount = pd.DataFrame(list(db.platcount.find({}, {'_id': 0})))

In [13]:
platcount.head()

,accession,count,man,organism
0,GPL15374,72,agilent,Thunnus thynnus
1,GPL15377,6,illumina,Alternaria brassicicola
2,GPL15378,7,agilent,Cottus
3,GPL15379,141,agilent,Canis lupus familiaris
4,GPL15380,92,illumina,Homo sapiens


In [10]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
query =  {
    "size": 0,
    "query": {"filtered": {
      "query": {"match_all": {}},
      "filter": {"bool": {
        "must": [
          {"term": {
            "data_source": "geo"
          }},
          {"term": {
            "organism": "Homo sapiens"
          }}
        ]
      }}
      
    }}, 
    "aggs" : {
        "platforms" : {
            "terms" : {
                "field" : "platform",
                "size": 4000
            }
            
        }
    }
}

res = es.search(index='samples_dev', body=query)


In [11]:
platcount_geo = (
    pd.DataFrame(res['aggregations']['platforms']['buckets'])
    .rename(columns={'key': 'accession'})
)

In [12]:
platcount_geo.head()

,doc_count,accession
0,108165,GPL570
1,36748,GPL96
2,33420,GPL10558
3,29069,GPL13534
4,23543,GPL6244


In [42]:
from elasticsearch import Elasticsearch
es = Elasticsearch()
query =  {
    "size": 0,
    "query": {"filtered": {
      "query": {"match_all": {}},
      "filter": {"bool": {
        "must": [
          {"term": {
            "data_source": "array-express"
          }},
          {"term": {
            "organism": "Homo sapiens"
          }}
        ]
      }}
      
    }}, 
    "aggs" : {
        "platforms" : {
            "terms" : {
                "field" : "platform",
                "size": 4000
            }
            
        }
    }
}

res = es.search(index='samples_dev', body=query)
platcount_ae = (
    pd.DataFrame(res['aggregations']['platforms']['buckets'])
    .rename(columns={'key': 'accession'})
)
platcount_ae.head()

,doc_count,accession
0,19111,A-AFFY-44
1,16378,A-AFFY-33
2,16325,A-MEXP-1171
3,10976,A-AGIL-11
4,8044,A-UMCU-3


In [53]:
platcount_ae.shape

(1083, 2)

In [ ]:
# pipeline = [
#     {'$match': {'data_source': 'geo'}},
#     {'$group': {'_id': '$platform', 'count': {'$sum': 1}}},
# ]
# platcount_geo = pd.DataFrame(list(db.samples.aggregate(pipeline)))

In [ ]:
# platcount_geo.shape()

In [ ]:
# platcount_geo.head()

In [43]:
platforms = pd.DataFrame(list(
    db.platforms.find({}, {
                'accession': 1,
                'title': 1,
                'data_source': 1,
                '_id': 0,
            })
    ))

In [14]:
platforms.head()

,accession,data_source,title
0,GPL15373,geo,Applied Biological Materials human miRNA (AMB ...
1,GPL15374,geo,Agilent-024425 Thunnus thynnus 2009 (Feature N...
2,GPL15377,geo,Illumina HiSeq 2000 (Alternaria brassicicola)
3,GPL15378,geo,Agilent-029126 order_cottus_expression_array_1...
4,GPL15379,geo,Agilent-021193 Canine (V2) Gene Expression Mic...


In [15]:
platforms_counts = platcount_geo.merge(platforms)

In [17]:
platforms_counts.shape

(3910, 4)

In [20]:
platforms_counts.head()

,doc_count,accession,data_source,title
0,108165,GPL570,geo,[HG-U133_Plus_2] Affymetrix Human Genome U133 ...
1,36748,GPL96,geo,[HG-U133A] Affymetrix Human Genome U133A Array
2,33420,GPL10558,geo,Illumina HumanHT-12 V4.0 expression beadchip
3,29069,GPL13534,geo,Illumina HumanMethylation450 BeadChip (HumanMe...
4,23543,GPL6244,geo,[HuGene-1_0-st] Affymetrix Human Gene 1.0 ST A...


In [19]:
platforms_counts.describe()

,doc_count
count,3910.000000
mean,234.123018
std,2206.726270
min,1.000000
25%,6.000000
50%,22.000000
75%,80.000000
max,108165.000000


In [22]:
platforms_counts.shape

(5618, 6)

In [21]:
platforms_counts.head()

,accession,count,man,organism,data_source,title
0,GPL15374,72,agilent,Thunnus thynnus,geo,Agilent-024425 Thunnus thynnus 2009 (Feature N...
1,GPL15377,6,illumina,Alternaria brassicicola,geo,Illumina HiSeq 2000 (Alternaria brassicicola)
2,GPL15378,7,agilent,Cottus,geo,Agilent-029126 order_cottus_expression_array_1...
3,GPL15379,141,agilent,Canis lupus familiaris,geo,Agilent-021193 Canine (V2) Gene Expression Mic...
4,GPL15380,92,illumina,Homo sapiens,geo,Illumina Sentrix Array Matrix (SAM) - GoldenGa...


In [25]:
FRMA_PLATFORMS = ['GPL' + str(x) for x in [
    570, 571, 96,
    1261,  # organism Mus musculus
    6244,  # getting strange errors. data in matrix
    # 1355,  # NO FRMA, Rattus norvegicus,
    6246,  # no frma package (?). read.affybatch: The affy package can process data from the Gene ST 1.x series of arrays,
    # but you should consider using either the oligo or xps packages, which are specifically
    # designed for these arrays.
    # there is no package called ‘mogene10stv1frmavecs’

    90, # organism Saccharomyces cerevisiae
    3921,
    # 5175, # no frma package (?). read.affybatch: The affy package is not designed for this array type. 
    # Please use either the oligo or xps package.
    # 5188, # no cdf frma packages
    8321,  # organism Mus musculus
]]

In [46]:
platforms_counts_ae = platcount_ae.merge(platforms).drop('data_source', axis=1)

In [51]:
platforms_counts.query('accession == "GPL570"').title.iloc[0]

'[HG-U133_Plus_2] Affymetrix Human Genome U133 Plus 2.0 Array'

In [52]:
platforms_counts_ae.query('accession == "A-AFFY-44"').title.iloc[0]

'Affymetrix GeneChip Human Genome U133 Plus 2.0 [HG-U133_Plus_2]'

In [40]:
(
    platforms_counts
    [platforms_counts.title.map(lambda t: 'methyl' not in t.lower())]
    .assign(**{'frma?': lambda d: d.accession.isin(FRMA_PLATFORMS).map(int)})
#     .set_index('accession') 
    .to_csv('../data/platforms-count-homo-geo.csv', index=False)
#     .sort('count', ascending=False)
#     .query('organism == "Homo sapiens"')
)

In [41]:
!wget http://www.ebi.ac.uk/arrayexpress/xml/v2/experiments?directsub=true

--2015-10-29 14:21:56--  http://www.ebi.ac.uk/arrayexpress/xml/v2/experiments?directsub=true
Resolving www.ebi.ac.uk (www.ebi.ac.uk)... 193.62.192.80
Connecting to www.ebi.ac.uk (www.ebi.ac.uk)|193.62.192.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘experiments?directsub=true’

    [                               <=>     ] 46,463,521  9.67MB/s   in 9.0s   

2015-10-29 14:22:07 (4.93 MB/s) - ‘experiments?directsub=true’ saved [46463521]

